This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/robot.html).  I recommend having both windows open, side-by-side!

In [ ]:
from pydrake.all import ModelVisualizer, StartMeshcat, PackageMap

from manipulation import running_as_notebook, ConfigureParser

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

# IIWA with a mobile base



In [ ]:
visualizer = ModelVisualizer(meshcat=meshcat)
ConfigureParser(visualizer.parser())
visualizer.AddModels(url="package://manipulation/mobile_iiwa14_primitive_collision.urdf")
visualizer.Run(loop_once=not running_as_notebook)
meshcat.DeleteAddedControls()